# Predicción de Volatilidad en Mercados Financieros: Un Enfoque Comparativo con Deep Learning (LSTM vs Transformer)

**Autores:** Juanjo y Raúl
**Asignatura:** Extensiones de Machine Learning - Bloque II
**Fecha:** Noviembre 2025

---

## 1. Resumen Ejecutivo

Este proyecto aborda el desafío de predecir los cambios en la volatilidad (*Delta Volatility*) de activos financieros, específicamente Apple Inc. (AAPL). A diferencia de los modelos econométricos tradicionales (como GARCH), proponemos un enfoque basado en datos (*data-driven*) utilizando arquitecturas de Deep Learning de vanguardia.

**Objetivos del Estudio:**
1.  **Modelado:** Implementar y optimizar dos arquitecturas profundas: **LSTM (Long Short-Term Memory)** y **Transformer** (basado en mecanismos de atención).
2.  **Robustez:** Diseñar un protocolo de entrenamiento riguroso que evite el sobreajuste y la fuga de datos (*data leakage*).
3.  **Evaluación Financiera:** Ir más allá de las métricas de error (MSE, R²) y evaluar la utilidad económica mediante una simulación de trading con gestión de riesgo dinámica.



**Hallazgo Clave:**
Los resultados experimentales demuestran la **superioridad de la arquitectura Transformer**. Gracias a su mecanismo de *Self-Attention*, este modelo no solo logra minimizar el error de predicción de manera más efectiva que la LSTM, sino que también ofrece una gestión de riesgos superior, generando un mayor *Sharpe Ratio* y reduciendo la exposición antes de eventos críticos de mercado ("cisnes negros").

In [ ]:
# IMPORTACIÓN DE LIBRERÍAS

# Librerías Estándar y Numéricas
import numpy as np
import pandas as pd
import json
import os
import warnings

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from IPython.display import display, HTML

# Preprocesamiento y Métricas
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

# Deep Learning (TensorFlow/Keras)
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Layer, Input, LSTM, Bidirectional, Dense, Dropout, 
    LayerNormalization, MultiHeadAttention, Add, 
    GlobalAveragePooling1D, Embedding
)
from tensorflow.keras.losses import Huber
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K # Importante para limpiar sesión
from scikeras.wrappers import KerasRegressor

# Configuración del Entorno
warnings.filterwarnings('ignore') # Ignorar advertencias no críticas
pd.set_option('display.max_columns', None) # Mostrar todas las columnas de pandas
pio.templates.default = "plotly_dark" # Tema oscuro para gráficos interactivos

print("Librerías importadas correctamente.")

Librerías importadas correctamente.


## 2. Ingeniería de Datos y Preprocesamiento

El éxito de un modelo de Deep Learning depende de la calidad de los datos. Utilizamos un dataset procesado de Apple Inc. que incluye características financieras derivadas.

**Decisiones de Diseño:**
*   **Variable Objetivo:** Predecimos el `target_delta_vol` ($Vol_{t+1} - Vol_{t}$). Esto estacionariza la serie, facilitando el aprendizaje.
*   **Ventana Temporal (`SEQUENCE_LENGTH`):** Utilizamos los últimos **30 días** de historia para predecir el siguiente paso.
*   **Volatilidad:** Utilizamos el estimador de **Garman-Klass**, que incorpora información intradía (High, Low, Open, Close), siendo más eficiente que la volatilidad basada solo en cierres.

In [ ]:
#  PARÁMETROS GLOBALES
ASSET = 'Apple (AAPL)'
FILE_PATH = './Datasets/Apple_processed.csv'

# Hiperparámetros de Datos
SEQUENCE_LENGTH = 30      # Ventana de observación (Lookback window)
TRAIN_VAL_SPLIT_PCT = 0.85 # 85% para Entrenamiento+Validación, 15% para Test

### 2.1. Carga y Estructuración Temporal del Dataset
En esta etapa, importamos el conjunto de datos que fue previamente limpiado y enriquecido con las variables financieras (como la volatilidad de Garman-Klass).

Un paso crítico en este bloque es la conversión y ordenamiento del índice temporal. Para modelos secuenciales como LSTM y Transformers, es imperativo que los datos estén estrictamente ordenados cronológicamente (sort_index). Cualquier desorden en las fechas podría introducir un sesgo de anticipación (look-ahead bias), donde el modelo aprendería accidentalmente de datos futuros durante la creación de las ventanas deslizantes.

In [ ]:
try:
    # Cargamos el dataset procesado desde la ruta relativa
    data_cleaned = pd.read_csv(FILE_PATH)

    # Normalización del nombre de la columna de fecha para robustez
    # Esto asegura que el código funcione tanto si la columna se llama 'date' como 'Date'
    date_col = None
    if 'date' in data_cleaned.columns: date_col = 'date'
    elif 'Date' in data_cleaned.columns: date_col = 'Date'

    if date_col:
        # Conversión explícita a formato datetime para manipulación de series temporales
        data_cleaned[date_col] = pd.to_datetime(data_cleaned[date_col])
        
        # Establecemos la fecha como índice del DataFrame
        data_cleaned.set_index(date_col, inplace=True)
        
        # Aseguramos que los datos estén ordenados del pasado al futuro
        # Esto es fundamental para evitar fugas de datos al crear las ventanas
        data_cleaned = data_cleaned.sort_index()
        
        print(f"Datos cargados para {ASSET}. Rango: {data_cleaned.index.min().date()} a {data_cleaned.index.max().date()}")
    else:
        print("ADVERTENCIA: No se encontró columna de fecha. El índice no es temporal")

    print(f"Dimensiones del dataset: {data_cleaned.shape}")

except FileNotFoundError:
    print(f"Error Crítico: Archivo no encontrado en {FILE_PATH}. Asegúrate de ejecutar el notebook de preparación primero")

Datos cargados para Apple (AAPL). Rango: 1980-12-19 a 2023-12-04
Dimensiones del dataset: (10830, 12)


### 2.2. Selección de Características y Normalización Robusta

Definimos la matriz de características ($X$) y la variable objetivo ($y$). Para garantizar la validez del experimento, aplicamos dos principios fundamentales:

1.  **División Temporal Estricta:** A diferencia de los problemas de clasificación estándar, aquí **no podemos barajar (shuffle)** los datos. El conjunto de prueba (*Test*) debe ser estrictamente posterior al de entrenamiento (*Train*) para simular un escenario de predicción real.
2.  **Prevención de Data Leakage:** Un error metodológico grave es calcular la media y desviación estándar sobre todo el dataset antes de dividirlo. Aquí ajustamos el `StandardScaler` **exclusivamente sobre el conjunto de entrenamiento**. Posteriormente, utilizamos esos parámetros estadísticos para transformar el conjunto de test. De esta forma, el modelo "no conoce" la distribución estadística del futuro.

In [ ]:
# Definición de predictores y variable objetivo
features = [
    'log_return', 'realized_vol_5d', 'return_range', 'volume_change', 'volatility'
]
target_col = 'target_delta_vol' 

# Creación de los subsets iniciales
X = data_cleaned[features]
y = data_cleaned[[target_col]] 

# Cálculo del índice de corte cronológico
total_samples = len(X)
train_val_end = int(total_samples * TRAIN_VAL_SPLIT_PCT)

# División secuencial (Train+Val vs Test)
X_train_val, y_train_val = X.iloc[:train_val_end], y.iloc[:train_val_end]
X_test, y_test = X.iloc[train_val_end:], y.iloc[train_val_end:]

print(f"Muestras de Entrenamiento+Validación: {len(X_train_val)}")
print(f"Muestras de Test (Hold-out): {len(X_test)}")

# Inicialización de escaladores independientes para X e y
scaler_x = StandardScaler()
scaler_y = StandardScaler()

# Ajuste (fit) y transformación sobre el conjunto de entrenamiento
X_train_val_scaled = scaler_x.fit_transform(X_train_val)
y_train_val_scaled = scaler_y.fit_transform(y_train_val)

# Transformación del test usando los parámetros aprendidos en entrenamiento
# No se hace fit() aquí para evitar filtración de información futura
X_test_scaled = scaler_x.transform(X_test) 
y_test_scaled = scaler_y.transform(y_test)

print("Datos estandarizados correctamente.")

Muestras de Entrenamiento+Validación: 9205
Muestras de Test (Hold-out): 1625
Datos estandarizados correctamente.


### 2.3. Generación de Estructuras Tensoriales (Windowing)

Las arquitecturas profundas como LSTM y Transformers no ingieren datos tabulares filas por columnas (2D). Requieren una estructura tridimensional: **[Muestras, Pasos de Tiempo, Características]**.

Para ello, aplicamos una técnica de **Ventana Deslizante** (*Sliding Window*):
1.  Recorremos la serie temporal paso a paso.
2.  En cada paso $t$, tomamos los **30 días previos** (`SEQUENCE_LENGTH`) como input ($X$).
3.  Asignamos el valor siguiente como objetivo ($y$).

Esto preserva la estructura secuencial local necesaria para que el mecanismo de atención y las puertas recurrentes detecten patrones temporales.

In [ ]:
# Función para generar secuencias temporales (Ventanas deslizantes)
def create_sequences(X, y, seq_length):
    """
    Transforma datos matriciales (2D) en tensores secuenciales (3D)
    Args:
        X: Variables predictoras escaladas
        y: Variable objetivo escalada
        seq_length: Longitud de la ventana de memoria (30 días)
    """
    X_seq, y_seq = [], []
    
    # Iteramos asegurando que haya espacio suficiente para la ventana
    for i in range(len(X) - seq_length):
        # Extraemos el bloque de historia (t-30 a t)
        X_seq.append(X[i : (i + seq_length)])
        
        # El target asociado a esta ventana
        # Nota: Ajustado al índice correspondiente según el preprocesamiento previo
        y_seq.append(y[i + seq_length - 1]) 
        
    return np.array(X_seq), np.array(y_seq)

# Generación de tensores 3D para Keras [Muestras, Pasos de Tiempo, Features]
X_train_val_seq, y_train_val_seq = create_sequences(X_train_val_scaled, y_train_val_scaled, SEQUENCE_LENGTH)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, SEQUENCE_LENGTH)

print(f"Tensor de Entrada (Train): {X_train_val_seq.shape} -> (Muestras, Ventana, Features)")
print(f"Tensor de Entrada (Test):  {X_test_seq.shape}")

# Recuperación de la escala original para evaluación
# Guardamos los valores reales (sin Z-Score) para calcular métricas financieras reales (no normalizadas)
y_test_real_unscaled = scaler_y.inverse_transform(y_test_seq)

Tensor de Entrada (Train): (9175, 30, 5) -> (Muestras, Ventana, Features)
Tensor de Entrada (Test):  (1595, 30, 5)


## 3. Diseño e Implementación de Modelos

### 3.1. Arquitectura Recurrente: LSTM Bidireccional

Las redes LSTM son el estándar en series temporales por su capacidad de mantener "memoria" a largo plazo, mitigando el problema del desvanecimiento del gradiente. Nuestra implementación busca robustez y generalización por encima de la complejidad excesiva.

**Decisiones de Diseño:**
* **Bidireccionalidad:** Procesamos la ventana temporal en ambas direcciones (pasado-futuro y futuro-pasado). Esto permite a la red capturar patrones de contexto completo antes de colapsar la información en la predicción.
* **Layer Normalization:** Aplicamos normalización *dentro* de las capas recurrentes. Esto es crítico en finanzas para estabilizar las activaciones internas frente a los cambios bruscos de varianza (heterocedasticidad).
* **Función de Pérdida Híbrida (Huber Loss):** En lugar de usar MSE puro, usamos Huber Loss. Esta función es cuadrática para errores pequeños (precisión) pero lineal para errores grandes (outliers), evitando que picos de volatilidad atípicos desestabilicen los pesos durante el entrenamiento.
* **Estabilidad Numérica (Gradient Clipping):** Configuramos el optimizador Adam con `clipnorm=1.0` para recortar los gradientes. Esto previene la "explosión de gradientes", un problema común en redes recurrentes profundas aplicadas a datos financieros volátiles.

In [ ]:
def build_lstm_model(lstm_units=50, dropout_rate=0.2, learning_rate=5e-4):
    """
    Construye y compila una red LSTM Bidireccional optimizada para regresión
    """
    n_features = 5 
    input_shape = (SEQUENCE_LENGTH, n_features)
    
    inputs = Input(shape=input_shape)
    
    # BLOQUE 1: Extracción de Características Secuenciales
    # return_sequences=True mantiene la dimensión temporal (Time Steps)
    x = Bidirectional(LSTM(units=lstm_units, return_sequences=True))(inputs)
    x = LayerNormalization()(x) # Normaliza activaciones por muestra (vital en series temporales)
    x = Dropout(dropout_rate)(x)
    
    # BLOQUE 2: Compresión de la Información
    # return_sequences=False colapsa la secuencia en un único vector de contexto
    x = Bidirectional(LSTM(units=lstm_units // 2, return_sequences=False))(x)
    x = LayerNormalization()(x)
    x = Dropout(dropout_rate)(x)
    
    # BLOQUE 3: Cabezal de Predicción (Head)
    x = Dense(units=32, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    
    # Salida: Un único valor continuo (Delta Volatility)
    outputs = Dense(units=1, activation='linear')(x)
    
    model = Model(inputs=inputs, outputs=outputs, name="LSTM_Model")
    
    # Compilación
    # Usamos clipnorm=1.0 para limitar la norma del gradiente y evitar explosiones numéricas
    model.compile(
        optimizer=Adam(learning_rate=learning_rate, clipnorm=1.0), 
        loss=Huber(), 
        metrics=['mae'] # Monitoreamos el error absoluto medio
    )
    return model

### 3.2. Modelo Transformer

Adaptamos la arquitectura Transformer para capturar relaciones complejas no lineales y dependencias de largo alcance mediante el mecanismo de **Self-Attention**.

**Decisiones Clave de Diseño:**
*   **Mecanismo de Atención:** Permite al modelo "enfocarse" selectivamente en los días de mayor relevancia informativa dentro de la ventana (ej. un shock de volatilidad hace 20 días), independientemente de su distancia temporal.
*   **Positional Encoding Sinusoidal:** Utilizamos una codificación matemática fija (senos y cosenos) en lugar de embeddings aprendibles. Esto inyecta una noción de tiempo relativo pura y evita el sobreajuste, lo cual es vital en datasets financieros donde la historia es limitada.
*   **Salida Autoregresiva:** Utilizamos exclusivamente el estado latente del **último token** tras el proceso de atención para proyectar la predicción, respetando estrictamente la causalidad temporal y la naturaleza del forecasting.

In [ ]:
# COMPONENTES DEL TRANSFORMER

class TransformerEncoder(Layer):
    """
    Bloque codificador estándar
    Utiliza mecanismos de atención y conexiones residuales para procesar la secuencia
    """
    def __init__(self, embed_dim, num_heads, ff_dim, dropout_rate=0.1, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.dropout_rate = dropout_rate
        
        # Mecanismo de Atención: Permite al modelo ponderar la importancia de cada paso temporal
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        
        # Red Feed-Forward interna
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"), 
            Dense(embed_dim)
        ])
        
        # Normalización por capas (crucial para convergencia en Transformers)
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(dropout_rate)
        self.dropout2 = Dropout(dropout_rate)
        
    def call(self, inputs, training=False):
        # 1. Sub-capa de Atención
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        
        # Conexión Residual (Add & Norm): inputs + attn_output
        # Esto permite que el gradiente fluya directamente, facilitando el entrenamiento profundo
        out1 = self.layernorm1(inputs + attn_output)
        
        # 2. Sub-capa Feed Forward
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        
        # Segunda Conexión Residual
        return self.layernorm2(out1 + ffn_output)
    
    def get_config(self):
        config = super().get_config()
        config.update({"embed_dim": self.embed_dim, "num_heads": self.num_heads,
                       "ff_dim": self.ff_dim, "dropout_rate": self.dropout_rate})
        return config

class SinusoidalPositionEncoding(Layer):
    """
    Inyecta información sobre la posición relativa de cada dato en la secuencia
    Usamos una fórmula matemática fija (senos/cosenos) en lugar de aprenderlos
    para evitar overfitting en datasets financieros pequeños
    """
    def __init__(self, seq_len, embed_dim, **kwargs):
        super(SinusoidalPositionEncoding, self).__init__(**kwargs)
        self.seq_len = seq_len
        self.embed_dim = embed_dim
        
    def call(self, x):
        x = tf.cast(x, tf.float32)
        
        # Matriz de posiciones (0, 1, 2, ..., seq_len)
        pos = tf.range(self.seq_len, dtype=tf.float32)[:, tf.newaxis]
        # Índices de dimensión para las frecuencias
        i = tf.range(self.embed_dim, dtype=tf.float32)[tf.newaxis, :]
        
        # Cálculo de frecuencias decrecientes
        angle_rates = 1 / tf.pow(10000.0, (2 * (i // 2)) / tf.cast(self.embed_dim, tf.float32))
        angle_rads = pos * angle_rates
        
        # Aplicamos patrón trigonométrico
        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])
        
        pos_encoding = tf.concat([sines, cosines], axis=-1)
        # Expandimos dimensión batch: (1, seq_len, embed_dim)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        
        # Sumamos la señal de posición a los datos originales
        return x + tf.cast(pos_encoding, x.dtype)
    
    def get_config(self):
        config = super().get_config()
        config.update({"seq_len": self.seq_len, "embed_dim": self.embed_dim})
        return config

def build_transformer_model(
    embed_dim=32, num_heads=2, ff_dim=32, dropout_rate=0.2, 
    learning_rate=5e-4, num_layers=1 
):
    n_features = 5 
    input_shape = (SEQUENCE_LENGTH, n_features)
    
    inputs = Input(shape=input_shape)
    
    # Proyección: Elevamos las features de entrada a un espacio latente de mayor dimensión
    x = Dense(embed_dim)(inputs) 
    
    # Sumamos la información de posición
    x = SinusoidalPositionEncoding(SEQUENCE_LENGTH, embed_dim)(x)
    
    # Apilamos bloques codificadores (N Layers)
    for _ in range(num_layers):
        x = TransformerEncoder(embed_dim, num_heads, ff_dim, dropout_rate)(x)
    
    # Global Average Pooling vs Last Token:
    # Seleccionamos SOLO el último paso de tiempo (t) para predecir (t+1).
    # Esto simula el estado más reciente de la memoria.
    x = x[:, -1, :]  
    
    # Cabezal de Regresión
    x = Dropout(dropout_rate)(x)
    x = Dense(16, activation='relu')(x)
    outputs = Dense(1, activation='linear')(x)
    
    model = Model(inputs=inputs, outputs=outputs, name="Transformer_Model")
    
    
    model.compile(
        optimizer=Adam(learning_rate=learning_rate, clipnorm=1.0), 
        loss=Huber(), 
        metrics=['mae']
    )
    return model

## 4. Optimización de Hiperparámetros

El rendimiento de una red neuronal es altamente sensible a su configuración. Para encontrar la arquitectura robusta y evitar decisiones arbitrarias, utilizamos **Grid Search** (Búsqueda en Rejilla), evaluando exhaustivamente múltiples combinaciones.

**Metodología de Validación:**

1.  **Validación Cruzada Temporal (`TimeSeriesSplit`):**
    A diferencia de la validación cruzada tradicional (K-Fold), en series temporales no podemos mezclar datos aleatoriamente. `TimeSeriesSplit` crea *folds* deslizantes donde el conjunto de validación siempre es **futuro** respecto al de entrenamiento, respetando la estricta causalidad temporal.

2.  **Criterio de Selección (`neg_mean_squared_error`):**
    Seleccionamos los modelos basándonos en la minimización del **Error Cuadrático Medio (MSE)**.
    * *Financieramente:* Preferimos penalizar de forma cuadrática los errores grandes. Un error pequeño es ruido aceptable, pero un fallo grande en la predicción de volatilidad puede implicar riesgos inasumibles.
    * *Técnicamente:* Scikit-Learn busca "maximizar" métricas, por lo que usamos el MSE negativo.




### 4.1. Optimización del Modelo Recurrente (LSTM)

En esta primera fase, exploramos el espacio de búsqueda para la red LSTM, centrándonos en equilibrar la memoria con la generalización:

* **Capacidad (`lstm_units`):** Probamos con 32 y 64 unidades para determinar cuánta "memoria" histórica es necesaria sin causar sobreajuste.
* **Regularización (`dropout_rate`):** Evaluamos tasas del 20% y 30% para "apagar" neuronas aleatoriamente y forzar a la red a aprender patrones robustos.
* **Convergencia (`learning_rate`):** Ajustamos la velocidad de aprendizaje del optimizador Adam.

In [ ]:
# Definimos la ruta para guardar/cargar los mejores parámetros (Persistencia)
PARAMS_FILE_LSTM = './Best_params/best_params_lstm.json'
best_hyperparams_lstm = {}

print("--- Iniciando Proceso de Optimización (LSTM) ---")

# 1. Definir el Wrapper de SciKeras
# Esto permite usar modelos de Keras como si fueran estimadores de Scikit-Learn
model_lstm = KerasRegressor(
    model=build_lstm_model,
    verbose=0 # Silenciamos el output de cada época para no saturar el notebook
)

# 2. Espacio de Búsqueda
# Nota: Los prefijos 'model__' mapean a los argumentos de la función build_lstm_model
param_grid_lstm = {
    'model__lstm_units': [32, 64],       # Capacidad de memoria
    'model__dropout_rate': [0.2, 0.3],   # Regularización
    'model__learning_rate': [1e-3, 5e-4],# Velocidad de convergencia
    'batch_size': [64],                  # Estabilidad del gradiente
    'epochs': [150]                      # Iteraciones fijas (suficientes para converger)
}

# 3. Configuración de Validación Cruzada
# n_splits=3 asegura validación en diferentes periodos del tiempo
tscv = TimeSeriesSplit(n_splits=3)

grid_lstm = GridSearchCV(
    estimator=model_lstm,
    param_grid=param_grid_lstm,
    cv=tscv,
    scoring='neg_mean_squared_error',
    n_jobs=1, # Usamos 1 proceso para evitar conflictos de memoria en GPU con TensorFlow
    verbose=1
)

# 4. Ejecución Condicional (Caching)
if os.path.exists(PARAMS_FILE_LSTM):
    # Si ya existe el fichero, cargamos los resultados para ahorrar tiempo
    print(f"Cargando parámetros pre-calculados desde '{PARAMS_FILE_LSTM}'")
    with open(PARAMS_FILE_LSTM, 'r') as f:
        best_hyperparams_lstm = json.load(f)
    print(f"Mejores Parámetros LSTM (Recuperados): {best_hyperparams_lstm}")
else:
    # Si no existe, ejecutamos el entrenamiento intensivo
    print(f"Ejecutando Grid Search completo para LSTM... (Esto puede tardar)")
    
    # Entrenamos explorando todas las combinaciones
    grid_result_lstm = grid_lstm.fit(X_train_val_seq, y_train_val_seq)
    
    print(f"Mejor Score (MSE Negativo): {grid_result_lstm.best_score_:.4f}")
    print(f"Mejores Parámetros: {grid_result_lstm.best_params_}")
    
    best_hyperparams_lstm = grid_result_lstm.best_params_
    
    # Guardamos en disco
    os.makedirs('./Best_params', exist_ok=True)
    with open(PARAMS_FILE_LSTM, 'w') as f:
        json.dump(best_hyperparams_lstm, f)
    print("Parámetros guardados correctamente.")

--- Iniciando Proceso de Optimización (LSTM) ---
Ejecutando Grid Search completo para LSTM...
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Mejor Score (MSE Negativo): -0.7009
Mejores Parámetros: {'batch_size': 64, 'epochs': 150, 'model__dropout_rate': 0.3, 'model__learning_rate': 0.0005, 'model__lstm_units': 32}
Parámetros guardados correctamente.


### 4.2. Optimización del Transformer

A diferencia de las redes recurrentes, el rendimiento del Transformer depende críticamente del equilibrio entre la capacidad de representación (dimensiones y capas) y la regularización.

Exploramos un espacio de búsqueda diseñado específicamente para series temporales financieras (donde los datos son escasos comparados con NLP):
* **Complejidad del Modelo:** Probamos diferentes tamaños para la red interna (`ff_dim`) y la profundidad (`num_layers`). Mantenemos las dimensiones de embedding contenidas (32) para evitar que el modelo memorice el ruido.
* **Mecanismo de Atención:** Configuramos `num_heads` para evaluar si dividir la atención en múltiples sub-espacios mejora la detección de patrones.

In [ ]:
# Definición de ruta para persistencia de mejores parámetros
PARAMS_FILE_TRANSFORMER = './Best_params/best_params_transformer.json'
best_hyperparams_transformer = {}

print("\n--- Iniciando Proceso de Optimización (Transformer) ---")

# Wrapper de Keras para Scikit-Learn
model_transformer = KerasRegressor(
    model=build_transformer_model,
    verbose=0
)

# Espacio de búsqueda específico para Transformer
# Nota: Mantenemos una arquitectura "ligera" (pocos heads/layers) para evitar overfitting
param_grid_transformer = {
    'model__embed_dim': [32],            # Dimensión del espacio latente
    'model__num_heads': [2],             # Cabezas de atención paralela
    'model__ff_dim': [64, 128],          # Capacidad de la red feed-forward interna
    'model__num_layers': [2],            # Profundidad de la red
    'model__dropout_rate': [0.2, 0.3],   # Regularización alta para datos ruidosos
    'model__learning_rate': [1e-3, 1e-4],# Tasas de aprendizaje estándar
    'batch_size': [64],
    'epochs': [150]
}

# Configuración del Grid Search con validación temporal
grid_transformer = GridSearchCV(
    estimator=model_transformer,
    param_grid=param_grid_transformer,
    cv=tscv,
    scoring='neg_mean_squared_error',
    n_jobs=1,
    verbose=1
)

# Lógica de Caching: Cargar si existe, Entrenar si no
if os.path.exists(PARAMS_FILE_TRANSFORMER):
    print(f"Cargando parámetros pre-calculados desde '{PARAMS_FILE_TRANSFORMER}'")
    with open(PARAMS_FILE_TRANSFORMER, 'r') as f:
        best_hyperparams_transformer = json.load(f)
    print(f"Mejores Parámetros Transformer (Recuperados): {best_hyperparams_transformer}")
else:
    print(f"Ejecutando Grid Search completo para Transformer...")
    # Ajuste del modelo
    grid_result_transformer = grid_transformer.fit(X_train_val_seq, y_train_val_seq)
    
    print(f"Mejor Score (MSE Negativo): {grid_result_transformer.best_score_:.4f}")
    print(f"Mejores Parámetros: {grid_result_transformer.best_params_}")
    
    best_hyperparams_transformer = grid_result_transformer.best_params_
    
    # Guardado de resultados
    os.makedirs('./Best_params', exist_ok=True)
    with open(PARAMS_FILE_TRANSFORMER, 'w') as f:
        json.dump(best_hyperparams_transformer, f)
    print("Parámetros guardados correctamente.")


--- Iniciando Proceso de Optimización (Transformer) ---
Ejecutando Grid Search completo para Transformer...
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Mejor Score (MSE Negativo): -0.6564
Mejores Parámetros: {'batch_size': 64, 'epochs': 150, 'model__dropout_rate': 0.2, 'model__embed_dim': 32, 'model__ff_dim': 64, 'model__learning_rate': 0.0001, 'model__num_heads': 2, 'model__num_layers': 2}
Parámetros guardados correctamente.


## 5. Protocolo de Entrenamiento Robusto

Antes de iniciar el entrenamiento, debemos preparar el entorno para garantizar la generalización y la estabilidad.

### 5.1. Subdivisión de Validación y Re-Escalado
Para que el *Early Stopping* funcione correctamente sin "quemar" el conjunto de Test final, realizamos una **subdivisión interna** en el conjunto de entrenamiento:

1.  **Validación Interna (80/20):** Separamos el último 20% de los datos de entrenamiento para monitorear la convergencia. El modelo nunca entrenará sobre estos datos; solo los usará para saber cuándo detenerse.
2.  **Prevención de Fuga de Datos (Re-Escalado):** Es un error común usar el escalador global aquí. Para mantener el rigor, ajustamos (`fit`) nuevos escaladores **exclusivamente** con el sub-conjunto de entrenamiento (80%) y transformamos la validación con esas estadísticas.

In [ ]:
# PREPARACIÓN DE DATOS PARA ENTRENAMIENTO FINAL (TRAIN / VALIDATION SPLIT)

# 1. Subdivisión Cronológica Interna
# Del conjunto de entrenamiento original (85% del total), separamos un 20% final 
# para usarlo como Validación (Early Stopping)
# NO TOCAMOS EL TEST SET FINAL.
train_val_len = len(X_train_val)
split_idx = int(train_val_len * 0.80)

# Datos Tabulares
X_train_part = X_train_val.iloc[:split_idx]
y_train_part = y_train_val.iloc[:split_idx]

X_val_part = X_train_val.iloc[split_idx:]
y_val_part = y_train_val.iloc[split_idx:]

print(f"Sub-Train (para ajustar pesos): {len(X_train_part)} muestras")
print(f"Validation (para Early Stopping): {len(X_val_part)} muestras")

# 2. Re-Escalado Estricto (Evitar Data Leakage)
# Es vital re-ajustar el scaler solo con el 'train_part' para que la media/std 
# de la validación no contamine el entrenamiento
scaler_x_final = StandardScaler()
scaler_y_final = StandardScaler()

X_train_part_scaled = scaler_x_final.fit_transform(X_train_part)
y_train_part_scaled = scaler_y_final.fit_transform(y_train_part)

# Transformamos validación usando estadísticas del sub-train
X_val_part_scaled = scaler_x_final.transform(X_val_part)
y_val_part_scaled = scaler_y_final.transform(y_val_part)

# 3. Generación de Secuencias (Tensores 3D)
X_train_seq_split, y_train_seq_split = create_sequences(X_train_part_scaled, y_train_part_scaled, SEQUENCE_LENGTH)
X_val_seq_split, y_val_seq_split = create_sequences(X_val_part_scaled, y_val_part_scaled, SEQUENCE_LENGTH)

print(f"\nTensor de Entrenamiento Final: {X_train_seq_split.shape}")
print(f"Tensor de Validación Final:    {X_val_seq_split.shape}")

Set de Entrenamiento Final: (7334, 30, 5)
Set de Validación Final:    (1811, 30, 5)


### 5.2. Estrategia de Estabilización (Best-of-N)

El entrenamiento de redes profundas es estocástico (depende de la semilla inicial). Para mitigar la suerte y asegurar resultados robustos, implementamos un bucle de **Entrenamiento Competitivo**:

1.  **Multi-Run:** Entrenamos cada arquitectura **5 veces** desde cero (limpiando sesión).
2.  **Callbacks Dinámicos:**
    * `EarlyStopping`: Detiene el entrenamiento si el error de validación no baja en 50 épocas.
    
    * `ReduceLROnPlateau`: Reduce la tasa de aprendizaje a la mitad si el modelo se estanca, permitiendo un ajuste fino ("fine-tuning") automático.
3.  **Selección del Ganador:** Conservamos únicamente la instancia que logre el mayor $R^2$ en el conjunto de validación.

In [ ]:
def train_best_model(model_builder, X_train, y_train, X_val, y_val, n_runs=5, epochs=150, batch_size=32, verbose=0):
    """
    Entrena el modelo múltiples veces y selecciona la mejor instancia (Best-of-N)
    para mitigar la aleatoriedad de la inicialización de pesos
    """
    best_model = None
    best_val_r2 = -float('inf') # Inicializamos con el peor score posible
    
    print(f"Iniciando protocolo de entrenamiento robusto ({n_runs} ejecuciones)")
    
    for i in range(n_runs):
        # LIMPIEZA DE SESIÓN (Crítico en bucles)
        # Borra el grafo computacional de Keras para asegurar que cada run empieza virgen
        K.clear_session() 
        
        # DEFINICIÓN DE CALLBACKS (Control dinámico)
        run_callbacks = [
            # Detener si no aprende nada nuevo en 50 épocas (evita overfitting)
            EarlyStopping(
                monitor='val_loss', 
                patience=50, 
                mode='min', 
                restore_best_weights=True # Volvemos al punto exacto antes de empezar a sobreajustar
            ),
            # Reducir velocidad de aprendizaje si nos estancamos
            ReduceLROnPlateau(
                monitor='val_loss', 
                factor=0.5,       # Reduce el LR a la mitad
                patience=15,      # Espera 15 épocas antes de actuar
                min_lr=1e-6,      # Límite inferior de seguridad
                verbose=0         
            )
        ]
        
        print(f"  > Ejecución {i+1}/{n_runs}...", end="")
        
        # Construimos una instancia nueva del modelo (pesos aleatorios nuevos)
        model = model_builder()
        
        # Entrenamos pasando los callbacks frescos
        history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=run_callbacks,
            shuffle=False, # Mantenemos orden temporal dentro del batch
            verbose=verbose
        )
        
        # Evaluación rápida del run actual
        y_val_pred = model.predict(X_val, verbose=0)
        val_r2 = r2_score(y_val, y_val_pred)
        print(f" R² Val: {val_r2:.4f}")
        
        # Lógica de Selección: ¿Es este run mejor que el anterior?
        if val_r2 > best_val_r2:
            best_val_r2 = val_r2
            best_model = model
            print(f"    [!] Nuevo mejor modelo encontrado (R²: {val_r2:.4f})")
            
    return best_model


# ENTRENAMIENTO LSTM
# Extraemos los hiperparámetros ganadores del Grid Search
final_params_lstm = {
    'lstm_units': best_hyperparams_lstm.get('model__lstm_units'),
    'dropout_rate': best_hyperparams_lstm.get('model__dropout_rate'),
    'learning_rate': best_hyperparams_lstm.get('model__learning_rate')
}

print("\n=== ENTRENANDO LSTM (Optimizado) ===")
# Usamos lambda para retrasar la creación del modelo hasta dentro del bucle
final_lstm_model = train_best_model(
    lambda: build_lstm_model(**final_params_lstm),
    X_train_seq_split, y_train_seq_split,
    X_val_seq_split, y_val_seq_split,
    n_runs=5, 
    batch_size=best_hyperparams_lstm.get('batch_size', 32)
)

# ENTRENAMIENTO TRANSFORMER
transformer_args = {
    'num_heads': best_hyperparams_transformer.get('model__num_heads'),
    'ff_dim': best_hyperparams_transformer.get('model__ff_dim'),
    'dropout_rate': best_hyperparams_transformer.get('model__dropout_rate'),
    'learning_rate': best_hyperparams_transformer.get('model__learning_rate'),
    'num_layers': best_hyperparams_transformer.get('model__num_layers'),
    'embed_dim': best_hyperparams_transformer.get('model__embed_dim'),
}

print("\n=== ENTRENANDO TRANSFORMER (Optimizado) ===")
final_transformer_model = train_best_model(
    lambda: build_transformer_model(**transformer_args),
    X_train_seq_split, y_train_seq_split,
    X_val_seq_split, y_val_seq_split,
    n_runs=5,
    batch_size=best_hyperparams_transformer.get('batch_size', 32)
)


=== ENTRENANDO LSTM (Optimizado) ===
Iniciando protocolo de entrenamiento robusto (5 ejecuciones)
  > Ejecución 1/5... R² Val: 0.2987
    [!] Nuevo mejor modelo encontrado (R²: 0.2987)
  > Ejecución 2/5... R² Val: 0.2596
  > Ejecución 3/5... R² Val: 0.2808
  > Ejecución 4/5... R² Val: 0.2756
  > Ejecución 5/5... R² Val: 0.3202
    [!] Nuevo mejor modelo encontrado (R²: 0.3202)

=== ENTRENANDO TRANSFORMER (Optimizado) ===
Iniciando protocolo de entrenamiento robusto (5 ejecuciones)
  > Ejecución 1/5... R² Val: 0.3199
    [!] Nuevo mejor modelo encontrado (R²: 0.3199)
  > Ejecución 2/5... R² Val: 0.3104
  > Ejecución 3/5... R² Val: 0.3108
  > Ejecución 4/5... R² Val: 0.3262
    [!] Nuevo mejor modelo encontrado (R²: 0.3262)
  > Ejecución 5/5... R² Val: 0.2914


## 6. Evaluación de Resultados y Comparativa Estadística

Llegamos a la fase crítica: evaluar el desempeño de los modelos en el conjunto de **Test (Hold-out)**. Estos datos corresponden a los últimos meses de la serie temporal y **nunca fueron vistos** por la red durante el entrenamiento ni la validación.

**Metodología de Evaluación:**
1.  **Inferencia Pura:** Realizamos predicciones sobre datos desconocidos.
2.  **Des-escalado:** Transformamos las predicciones (que están en rango normalizado) a su escala original. Esto es vital para saber cuánto nos equivocamos en términos reales de volatilidad.
3.  **El "Baseline Ingenuo" (Benchmark):** Comparamos nuestros modelos sofisticados contra una heurística simple: asumir que la volatilidad de mañana será igual a la de hoy ($\Delta Vol = 0$).
    * Si $R^2_{Modelo} > R^2_{Baseline}$, el modelo aporta valor predictivo real.
    * Si $R^2_{Modelo} \le R^2_{Baseline}$, el modelo es ruido y no sirve.

In [ ]:
def evaluate_model_with_baseline(model, X_test, y_test_unscaled, model_name):
    print(f"\n--- Evaluación: {model_name} ---")
    
    # Inferencia sobre datos de Test (nunca vistos)
    y_pred_scaled = model.predict(X_test, verbose=0)
    
    # Des-escalamos para tener errores en unidades reales de volatilidad
    # Importante para comparar magnitud real de error
    y_pred_unscaled = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))
    
    # Métricas del Modelo
    r2 = r2_score(y_test_unscaled, y_pred_unscaled)
    mae = mean_absolute_error(y_test_unscaled, y_pred_unscaled)
    rmse = np.sqrt(mean_squared_error(y_test_unscaled, y_pred_unscaled))
    
    print(f"Resultados {model_name}:")
    print(f"  R²:   {r2:.4f}")
    print(f"  MAE:  {mae:.6f}")
    print(f"  RMSE: {rmse:.6f}")
    
    # 3. Baseline (Predicción Constante = 0)
    # Asumimos la hipótesis de mercado eficiente: el mejor predictor es el valor actual (cambio 0)
    y_pred_baseline = np.zeros_like(y_test_unscaled)
    r2_base = r2_score(y_test_unscaled, y_pred_baseline)
    
    print(f"Resultados Baseline:")
    print(f"  R²:   {r2_base:.4f}")
    
    # 4. Conclusión
    if r2 > r2_base:
        print(f"El modelo supera al baseline")
    else:
        print(f"El modelo no supera al baseline.")
    
    return y_pred_unscaled, r2, mae, rmse

# Evaluación LSTM
y_pred_lstm, r2_lstm, mae_lstm, rmse_lstm = evaluate_model_with_baseline(
    final_lstm_model, X_test_seq, y_test_real_unscaled, "LSTM"
)

# Evaluación Transformer
y_pred_transformer, r2_transformer, mae_transformer, rmse_transformer = evaluate_model_with_baseline(
    final_transformer_model, X_test_seq, y_test_real_unscaled, "Transformer"
)


--- Evaluación: LSTM ---
Resultados LSTM:
  R²:   0.2780
  MAE:  0.003872
  RMSE: 0.005566
Resultados Baseline:
  R²:   -0.0000
El modelo supera al baseline

--- Evaluación: Transformer ---
Resultados Transformer:
  R²:   0.2841
  MAE:  0.003769
  RMSE: 0.005543
Resultados Baseline:
  R²:   -0.0000
El modelo supera al baseline


### 6.1. Análisis Visual del Comportamiento y Precisión Direccional

Más allá de las métricas de error global (como el RMSE), en series temporales financieras es crítico inspeccionar **cómo** se equivoca el modelo.
[Image of stock market volatility chart]
Generamos una visualización interactiva para responder a dos preguntas cualitativas:
1.  **¿Existe Retraso (Lag)?** Las redes recurrentes clásicas tienden a "perseguir" el precio, reaccionando un día tarde a los picos de volatilidad. Buscamos verificar visualmente si el mecanismo de atención del Transformer mitiga este efecto.
2.  **¿Acierta la Dirección?** Calculamos explícitamente la *Directional Accuracy*: el porcentaje de veces que el modelo predice correctamente el signo del cambio (subida o bajada), una métrica a menudo más relevante para el trading que la precisión decimal exacta.

In [ ]:
# VISUALIZACIÓN COMPARATIVA LSTM vs TRANSFORMER
# Utilizando Plotly para generar gráficos interactivos con zoom y slider temporal

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

def plot_final_comparison(y_real, pred_lstm, pred_trans, dates):
    # Aplanamos arrays para asegurar compatibilidad con Plotly (de 2D a 1D)
    y_real = y_real.flatten()
    pred_lstm = pred_lstm.flatten() if pred_lstm is not None else None
    pred_trans = pred_trans.flatten() if pred_trans is not None else None
    
    # Cálculo de Métricas de Acierto en Dirección (Directional Accuracy)
    # Comparamos si el signo de la predicción coincide con el signo real
    stats_text = ""
    if pred_lstm is not None:
        da_lstm = np.mean(np.sign(y_real) == np.sign(pred_lstm)) * 100
        stats_text += f"Acierto Dirección LSTM: {da_lstm:.1f}% | "
    if pred_trans is not None:
        da_trans = np.mean(np.sign(y_real) == np.sign(pred_trans)) * 100
        stats_text += f"Acierto Dirección Trans: {da_trans:.1f}%"

    fig = go.Figure()

    # 1. La Realidad (Ground Truth)
    fig.add_trace(go.Scatter(
        x=dates, y=y_real,
        mode='lines', name='Realidad',
        line=dict(color='rgba(255, 255, 255, 0.3)', width=1),
        fill='tozeroy', fillcolor='rgba(255, 255, 255, 0.05)' # Sombreado de fondo para contexto
    ))

    # 2. Predicción LSTM (Azul Cían)
    if pred_lstm is not None:
        fig.add_trace(go.Scatter(
            x=dates, y=pred_lstm,
            mode='lines', name='LSTM',
            line=dict(color='#00d2ff', width=2)
        ))

    # 3. Predicción Transformer (Naranja)
    if pred_trans is not None:
        fig.add_trace(go.Scatter(
            x=dates, y=pred_trans,
            mode='lines', name='Transformer',
            line=dict(color='#ff9f43', width=2)
        ))

    # Configuración del Layout Interactivo
    fig.update_layout(
        title=f"<b>Comparativa de Modelos: Predicción de Delta Volatilidad</b><br><span style='font-size:12px'>{stats_text}</span>",
        yaxis_title="Delta Volatilidad (Cambio diario)",
        xaxis_title="Fecha",
        template="plotly_dark",
        height=600,
        legend=dict(orientation="h", y=1.02, xanchor="right", x=1),
        hovermode="x unified", # Facilita comparar valores al pasar el ratón
        xaxis=dict(
            rangeslider=dict(visible=True), # Slider inferior para navegar en el tiempo
            type="date"
        )
    )
            
    fig.show()

print("\n--- GENERANDO COMPARATIVA FINAL ---")
# Verificamos qué modelos existen en memoria para evitar errores si alguno falló
plstm = y_pred_lstm if 'y_pred_lstm' in globals() else None
ptrans = y_pred_transformer if 'y_pred_transformer' in globals() else None

# Generamos el plot usando las fechas correspondientes al conjunto de Test
plot_final_comparison(y_test_real_unscaled, plstm, ptrans, data_cleaned.index[train_val_end + SEQUENCE_LENGTH:])


--- GENERANDO COMPARATIVA FINAL ---


## 7. Simulación de Trading y Prueba de Robustez en Oro (Gold)

Un modelo puede tener un buen ajuste estadístico ($R^2$) pero ser inútil para la gestión de carteras. Para validar la utilidad económica real, implementamos un motor de **Backtesting con Gestión Dinámica de Riesgo**.

### 7.1. Estrategia: "Adaptive Volatility Targeting"
No buscamos predecir la dirección del precio (subida/bajada), sino **modular la exposición** al activo en función del riesgo percibido.
* **Algoritmo:** Si el modelo predice un aumento de la volatilidad ($\Delta Vol > 0$), el sistema reduce la posición proporcionalmente. Si la predicción supera un "Umbral de Pánico" (basado en el percentil 95 histórico), la estrategia sale totalmente a liquidez (Cash).
1.  **Predicción:** El modelo estima si la volatilidad subirá ($\Delta > 0$) o bajará ($\Delta \le 0$).
2.  **Decisión:**
    *   Si se predice **Calma** ($\Delta \le 0$): Mantenemos **100% invertidos**.
    *   Si se predice **Turbulencia** ($\Delta > 0$): Reducimos la exposición proporcionalmente. Si la predicción supera un "Umbral de Pánico" (percentil 95 histórico), salimos totalmente a Cash (0% exposición).
* **Objetivo:** Suavizar la curva de capital y evitar grandes caídas (*Drawdowns*).

$$ Exposición_t = \max(0, 1 - \frac{\text{Predicción}_t}{\text{Umbral Pánico}}) $$

### 7.2. Generalización: El Test del Oro
Para evitar sesgos y demostrar que nuestros modelos han aprendido patrones universales de volatilidad y no solo "memorizado" el gráfico de Apple, realizamos la prueba de fuego en un mercado totalmente distinto: **Futuros del Oro (Gold)**.
[Image of gold bars market chart]
Aplicamos la misma arquitectura y lógica sobre este *commodity*. Si el Transformer es realmente superior capturando dinámicas no lineales, debería superar a la LSTM y al *Buy & Hold* también en este nuevo entorno, protegiendo el capital durante las semanas de crisis.

In [ ]:
## 7. Simulación de Trading: De la Estadística a la Economía

class AdaptiveExposureSimulator:
    """
    Simulador de gestión de cartera que ajusta la exposición (0% a 100%)
    basándose en la severidad de la volatilidad predicha.
    """
    def __init__(self, initial_capital, risk_percentile=95):
        self.initial_capital = initial_capital
        self.risk_percentile = risk_percentile # Sensibilidad al riesgo (Umbral de Pánico)
        self.max_risk_delta = None 

    def calibrate(self, pred_deltas):
        """Autocalibración: Define qué se considera una 'volatilidad extrema' en este mercado"""
        positive_deltas = pred_deltas[pred_deltas > 0]
        if len(positive_deltas) == 0:
            self.max_risk_delta = 1.0
            return
        # El umbral es el percentil 95 de los aumentos históricos predichos
        self.max_risk_delta = np.percentile(positive_deltas, self.risk_percentile)
        self.max_risk_delta = max(self.max_risk_delta, 1e-5)

    def run(self, pred_deltas_unscaled, real_returns):
        if self.max_risk_delta is None:
            self.calibrate(pred_deltas_unscaled)
            
        capital = self.initial_capital
        portfolio_values = [capital]
        exposures = [0]
        
        min_len = min(len(pred_deltas_unscaled), len(real_returns))
        
        for i in range(min_len):
            pred_delta = pred_deltas_unscaled[i]
            
            # LÓGICA DE GESTIÓN DE RIESGO
            if pred_delta <= 0:
                # Zona Segura (Volatilidad bajando): Exposición Total
                exposure = 1.0
            else:
                # Zona de Riesgo (Volatilidad subiendo): Reducción Proporcional
                # Cuanto mayor sea la predicción frente al umbral, menos invertimos
                ratio = pred_delta / self.max_risk_delta
                exposure = 1.0 - ratio
            
            # Limitamos la exposición entre 0 (Cash) y 1 (Full Invested)
            exposure = np.clip(exposure, 0.0, 1.0)
            
            # Aplicamos el retorno del día ponderado por la exposición
            ret = real_returns[i]
            capital = capital * (1 + ret * exposure)
            
            portfolio_values.append(capital)
            exposures.append(exposure)
            
        return np.array(portfolio_values), np.array(exposures)

def calculate_metrics(portfolio_values, initial_capital, name):
    """Calcula KPIs financieros estándar: Rentabilidad, Sharpe y Drawdown"""
    returns = pd.Series(portfolio_values).pct_change().dropna()
    total_return = (portfolio_values[-1] - initial_capital) / initial_capital
    
    if len(returns) > 0:
        volatility = returns.std() * np.sqrt(252) # Anualizada
        # Ratio de Sharpe (asumiendo tasa libre de riesgo 0 para simplificar)
        sharpe = (returns.mean() * 252) / volatility if volatility > 0 else 0
        
        # Max Drawdown (Peor caída desde un máximo histórico)
        cum_returns = np.array(portfolio_values)
        peak = np.maximum.accumulate(cum_returns)
        drawdown = (cum_returns - peak) / peak
        max_drawdown = drawdown.min()
    else:
        sharpe = max_drawdown = 0.0
        
    return {
        'Estrategia': name,
        'Retorno Total': total_return,
        'Sharpe Ratio': sharpe,
        'Max Drawdown': max_drawdown
    }


print("\n=== INICIANDO SIMULACIÓN DE GENERALIZACIÓN: MERCADO DEL ORO ===")

# CARGA Y PREPROCESAMIENTO ESPECÍFICO PARA ORO
GOLD_FILE_PATH = './Datasets/Gold_processed.csv'
try:
    gold_data = pd.read_csv(GOLD_FILE_PATH)
    
    # Normalización de fechas e índice
    date_col_gold = None
    if 'date' in gold_data.columns: date_col_gold = 'date'
    elif 'Date' in gold_data.columns: date_col_gold = 'Date'
    
    if date_col_gold:
        gold_data[date_col_gold] = pd.to_datetime(gold_data[date_col_gold])
        gold_data.set_index(date_col_gold, inplace=True)
        gold_data = gold_data.sort_index()
        print(f"Datos Gold cargados. Rango: {gold_data.index.min().date()} a {gold_data.index.max().date()}")
    
    # features_gold = [...] (Mismas columnas que en Apple)
    features_gold = ['log_return', 'realized_vol_5d', 'return_range', 'volume_change', 'volatility']
    target_col_gold = 'target_delta_vol'
    
    X_gold = gold_data[features_gold]
    y_gold = gold_data[[target_col_gold]]
    
    # RE-ESCALADO
    # Ajustamos (fit) un escalador NUEVO a los datos de Oro.
    # Los modelos esperan una distribución N(0,1), pero la media/std del Oro 
    # es distinta a la de Apple. Si usáramos el scaler de Apple, distorsionaríamos los datos.
    scaler_x_gold = StandardScaler()
    X_gold_scaled = scaler_x_gold.fit_transform(X_gold)
    
    scaler_y_gold = StandardScaler()
    y_gold_scaled = scaler_y_gold.fit_transform(y_gold)
    
    # GENERACIÓN DE SECUENCIAS
    X_gold_seq, y_gold_seq = create_sequences(X_gold_scaled, y_gold_scaled, SEQUENCE_LENGTH)
    print(f"Secuencias Gold generadas: {X_gold_seq.shape}")
    
    # INFERENCIA CON MODELOS PRE-ENTRENADOS
    # Aplicamos la "Inteligencia" aprendida en Apple sobre el Oro
    y_pred_lstm_gold_scaled = final_lstm_model.predict(X_gold_seq, verbose=0)
    y_pred_trans_gold_scaled = final_transformer_model.predict(X_gold_seq, verbose=0)
    
    # Recuperamos la magnitud real de la volatilidad del Oro
    y_pred_lstm_gold = scaler_y_gold.inverse_transform(y_pred_lstm_gold_scaled)
    y_pred_trans_gold = scaler_y_gold.inverse_transform(y_pred_trans_gold_scaled)
    
    # EJECUCIÓN DEL BACKTEST
    real_returns_gold = gold_data['log_return'].iloc[SEQUENCE_LENGTH:].values
    dates_gold_sim = gold_data.index[SEQUENCE_LENGTH:]
    
    # Alineación de longitudes (por si el batching recortó el final)
    min_len_gold = min(len(y_pred_lstm_gold), len(real_returns_gold))
    
    pred_lstm_gold_sim = y_pred_lstm_gold[:min_len_gold].flatten()
    pred_trans_gold_sim = y_pred_trans_gold[:min_len_gold].flatten()
    real_returns_gold_sim = real_returns_gold[:min_len_gold]
    dates_gold_sim = dates_gold_sim[:min_len_gold]
    
    # Instanciamos simuladores independientes
    INITIAL_CAPITAL = 10000
    sim_lstm_gold = AdaptiveExposureSimulator(INITIAL_CAPITAL)
    sim_trans_gold = AdaptiveExposureSimulator(INITIAL_CAPITAL)
    
    vals_lstm_gold, exp_lstm_gold = sim_lstm_gold.run(pred_lstm_gold_sim, real_returns_gold_sim)
    vals_trans_gold, exp_trans_gold = sim_trans_gold.run(pred_trans_gold_sim, real_returns_gold_sim)

    # Benchmark: Buy & Hold (Mantenerse comprado siempre)
    vals_bh_gold = [INITIAL_CAPITAL]
    for r in real_returns_gold_sim:
        vals_bh_gold.append(vals_bh_gold[-1] * (1 + r))
    vals_bh_gold = np.array(vals_bh_gold)
    
    # Visualización Interactiva (Plotly)
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                        subplot_titles=("Evolución del Capital (Equity Curve)", "Exposición al Riesgo"))
    
    fig.add_trace(go.Scatter(x=dates_gold_sim, y=vals_lstm_gold[1:], name='LSTM Equity', line=dict(color='Blue')), row=1, col=1)
    fig.add_trace(go.Scatter(x=dates_gold_sim, y=vals_trans_gold[1:], name='Transformer Equity', line=dict(color='Red')), row=1, col=1)
    fig.add_trace(go.Scatter(x=dates_gold_sim, y=vals_bh_gold[1:], name='Buy & Hold', line=dict(color='white', dash='dash')), row=1, col=1)
    
    fig.add_trace(go.Scatter(x=dates_gold_sim, y=exp_lstm_gold[1:], name='Exp LSTM', line=dict(color='Blue', width=1), showlegend=False), row=2, col=1)
    fig.add_trace(go.Scatter(x=dates_gold_sim, y=exp_trans_gold[1:], name='Exp Transformer', line=dict(color='Red', width=1), showlegend=False), row=2, col=1)
    
    fig.update_layout(title="<b>Backtesting en ORO: LSTM vs Transformer vs Buy & Hold</b>", height=800, template="plotly_dark")
    fig.show()
    
    # Tabla de Resultados Financieros
    metrics = [
        calculate_metrics(vals_lstm_gold, INITIAL_CAPITAL, 'LSTM'),
        calculate_metrics(vals_trans_gold, INITIAL_CAPITAL, 'Transformer'),
        calculate_metrics(vals_bh_gold, INITIAL_CAPITAL, 'Buy & Hold')
    ]
    df_metrics_gold = pd.DataFrame(metrics).set_index('Estrategia')
    display(df_metrics_gold.style.format({
        'Retorno Total': '{:.2%}',
        'Sharpe Ratio': '{:.2f}',
        'Max Drawdown': '{:.2%}'
    }).background_gradient(cmap='RdYlGn'))

except Exception as e:
    print(f"Error en simulación: {e}")


print("\n=== PRUEBA DEL CRASH (SEMANAL): ANÁLISIS DE COMPORTAMIENTO EN SEMANAS BAJISTAS ===")

try:
    if 'real_returns_gold_sim' in locals():
        target_returns = real_returns_gold_sim
        target_dates = dates_gold_sim
        target_exp_lstm = exp_lstm_gold
        target_exp_trans = exp_trans_gold
        market_name = "Gold"
    else:
        raise ValueError("No se encontraron datos de simulación (Gold).")

    # Análisis Forense: ¿Qué hicieron los modelos en las peores semanas?
    df_daily = pd.DataFrame({
        'Return': target_returns,
        'Exp_LSTM': target_exp_lstm[1:], 
        'Exp_Transformer': target_exp_trans[1:]
    }, index=target_dates)
    
    # Agrupamos por semana para filtrar el ruido diario y ver tendencias de crisis
    df_weekly = df_daily.resample('W-FRI').agg({
        'Return': 'sum',      # El retorno se acumula
        'Exp_LSTM': 'mean',   # La exposición se promedia
        'Exp_Transformer': 'mean'
    })
    
    # Seleccionamos las 5 peores caídas del periodo
    N_WORST_WEEKS = 5
    worst_weeks = df_weekly.sort_values('Return').head(N_WORST_WEEKS)
    
    print(f"\nTop {N_WORST_WEEKS} Peores Semanas en {market_name} y Exposición Promedio:")
    display(worst_weeks.style.format({
        'Return': '{:.2%}',
        'Exp_LSTM': '{:.2%}',
        'Exp_Transformer': '{:.2%}'
    }).background_gradient(subset=['Return'], cmap='RdYlGn')
      .background_gradient(subset=['Exp_LSTM', 'Exp_Transformer'], cmap='Reds'))
    
    # Comparación de "Reflejos"
    avg_exp_lstm_crash = worst_weeks['Exp_LSTM'].mean()
    avg_exp_trans_crash = worst_weeks['Exp_Transformer'].mean()
    
    print(f"\nExposición Promedio durante las {N_WORST_WEEKS} peores semanas:")
    print(f"  LSTM:        {avg_exp_lstm_crash:.2%}")
    print(f"  Transformer: {avg_exp_trans_crash:.2%}")
    
    diff = avg_exp_lstm_crash - avg_exp_trans_crash
    if diff > 0:
        print(f"\nCONCLUSIÓN: En semanas de crisis, el Transformer redujo su exposición un {diff:.2%} más que el LSTM.")
    else:
        print(f"\nCONCLUSIÓN: El Transformer estuvo un {abs(diff):.2%} más expuesto que el LSTM en estas semanas.")

except Exception as e:
    print(f"No se pudo realizar el análisis semanal: {e}")


=== INICIANDO SIMULACIÓN EN MERCADO DEL ORO ===
Datos Gold cargados. Rango: 2011-12-22 a 2018-12-28
Secuencias Gold generadas: (1682, 30, 5)


,Retorno Total,Sharpe Ratio,Max Drawdown
Estrategia,,,
LSTM,-22.21%,-0.22,-32.22%
Transformer,-16.72%,-0.14,-30.28%
Buy & Hold,-33.99%,-0.33,-45.12%



=== PRUEBA DEL CRASH (SEMANAL): ANÁLISIS DE COMPORTAMIENTO EN SEMANAS BAJISTAS ===

Top 5 Peores Semanas en Gold y Exposición Promedio:


,Return,Exp_LSTM,Exp_Transformer
Date,,,
2013-04-19 00:00:00,-9.19%,100.00%,100.00%
2013-06-21 00:00:00,-7.23%,76.04%,60.36%
2013-05-17 00:00:00,-6.30%,95.73%,80.00%
2013-04-12 00:00:00,-5.97%,99.78%,100.00%
2014-10-31 00:00:00,-4.93%,71.15%,61.85%



Exposición Promedio durante las 5 peores semanas:
  LSTM:        88.54%
  Transformer: 80.44%

CONCLUSIÓN: En semanas de crisis, el Transformer redujo su exposición un 8.10% más que el LSTM.
